In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from torchtext.legacy.data import Field, TabularDataset, BucketIterator,ReversibleField
import matplotlib.pyplot as plt
from ast import literal_eval
import remi_utils as utils
import twoencodertransformer as kk
import pickle
source_folder = "solo_generation_dataset_fixed"
folder = "fixed_models/intro"
destination_folder = folder + "/solo_generation_weights"
generated_outputs = folder +  "/generated_samples"
vocab = folder + "/vocab"

In [2]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

state = pickle.load(open('./state.pkl', 'rb'))
random.setstate(state)

In [3]:
from pathlib import Path
Path(destination_folder).mkdir(parents=True, exist_ok=True)
Path(generated_outputs).mkdir(parents=True, exist_ok=True)
Path(vocab).mkdir(parents=True, exist_ok=True)
Path(generated_outputs+"/intro").mkdir(parents=True, exist_ok=True)
Path(generated_outputs+"/outro").mkdir(parents=True, exist_ok=True)
Path(generated_outputs+"/solo").mkdir(parents=True, exist_ok=True)
Path(generated_outputs+"/predict").mkdir(parents=True, exist_ok=True)

In [4]:
event2word, word2event = pickle.load(open('dictionary_fixed.pkl', 'rb'))

In [5]:
if torch.cuda.is_available():  
    dev = "cuda:1" 
else:  
    dev = "cpu" 
print(dev)
device = torch.device(dev)
print(device)

cuda:1
cuda:1


In [6]:
# Fields

intro_field = Field(tokenize=None, lower=True, include_lengths=True, batch_first=True, init_token="<sos>", eos_token="<eos>")
intro_piano_field = Field(tokenize=None, lower=True, include_lengths=True, batch_first=True, init_token="<sos>", eos_token="<eos>")
outro_field = Field(tokenize=None, lower=True, include_lengths=True, batch_first=True, init_token="<sos>", eos_token="<eos>")
outro_piano_field = Field(tokenize=None, lower=True, include_lengths=True, batch_first=True, init_token="<sos>", eos_token="<eos>")
solo_field = Field(tokenize=None, lower=True, include_lengths=True, batch_first=True, init_token="<sos>", eos_token="<eos>")
solo_piano_field = Field(tokenize=None, lower=True, include_lengths=True, batch_first=True, init_token="<sos>", eos_token="<eos>")
fields = [('intro', intro_field), ('intro_piano', intro_piano_field), \
          ('outro', outro_field), ('outro_piano', outro_piano_field), \
          ('solo', solo_field), ('solo_piano', solo_piano_field)]

# TabularDataset

train, valid, test = TabularDataset.splits(path=source_folder, train='train_torchtext.csv', validation='val_torchtext.csv', test='test_torchtext.csv',
                                           format='CSV', fields=fields, skip_header=True)

# Iterators
BATCH_SIZE = 8
train_iter = BucketIterator(train, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.intro),
                            device=device, sort=False, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.intro),
                            device=device, sort=False, sort_within_batch=True)
test_iter = BucketIterator(test, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.intro),
                            device=device, sort=False, sort_within_batch=True)

# Vocabulary

intro_field.build_vocab(train, min_freq=1)
intro_piano_field.build_vocab(train, min_freq=1)
outro_field.build_vocab(train, min_freq=1)
outro_piano_field.build_vocab(train, min_freq=1)
solo_field.build_vocab(train, min_freq=1)
solo_piano_field.build_vocab(train, min_freq=1)

In [7]:
for ((intro, intro_len), (intro_piano, intro_piano_len),\
     (outro, outro_len),(outro_piano, outro_piano_len),\
     (solo, solo_len),(solo_piano, solo_piano_len)), _ in (test_iter):
    print(solo.transpose(1,0).size())

torch.Size([269, 8])
torch.Size([331, 8])
torch.Size([353, 8])
torch.Size([522, 8])
torch.Size([325, 8])
torch.Size([444, 8])
torch.Size([281, 8])
torch.Size([473, 8])
torch.Size([619, 8])
torch.Size([365, 8])
torch.Size([509, 8])
torch.Size([353, 8])
torch.Size([433, 8])
torch.Size([491, 8])


In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
torch.backends.cudnn.enabled=False

In [9]:
#https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/more_advanced/seq2seq_transformer/seq2seq_transformer.py
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )
        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )

        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask,
        )
        out = self.fc_out(out)
        return out


In [10]:
src_vocab_size = len(intro_field.vocab)
trg_vocab_size = len(solo_field.vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.10
max_len = 1200
forward_expansion = 4
src_pad_idx = 1 #english.vocab.stoi["<pad>"]

model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
)
model = model.to(device)

In [11]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters(), lr=2e-4) #4e-5


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')


def save_best_checkpoint(state, nth,filename="_checkpoint.pt"):
    print("=> Saving checkpoint")
#     torch.save(state, destination_folder + str(nth)+filename)
    torch.save(state, destination_folder + '/metrics.pt')

def save_final_checkpoint(state, nth,filename="_checkpoint.pt"):
    print("=> Saving checkpoint")
    torch.save(state, destination_folder + "/" + str(nth)+filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

The model has 11,076,868 trainable parameters


In [12]:
# stoi input str get int
# intro_field.vocab.stoi
# itos input into get token/str
# intro_field.vocab.itos[4]

In [13]:
PAD_IDX = 1

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
#criterion = nn.CrossEntropyLoss()

In [14]:
import math
import time


def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    #for _, (src, _,trg,_) in enumerate(iterator):
    for ((intro, intro_len), (intro_piano, intro_piano_len),\
     (outro, outro_len),(outro_piano, outro_piano_len),\
     (solo, solo_len),(solo_piano, solo_piano_len)), _ in (iterator):
        src, trg = intro.transpose(1,0), solo.transpose(1,0)
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()
        output = model(src, trg[:-1, :])
        
#         print(output.size())
#         print(trg.size())
        
        output = output.view(-1, output.shape[-1])
        trg = trg[1:].reshape(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.cpu().detach().item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        #for _, (src, _,trg,_) in enumerate(iterator):
        for ((intro, intro_len), (intro_piano, intro_piano_len),\
         (outro, outro_len),(outro_piano, outro_piano_len),\
         (solo, solo_len),(solo_piano, solo_piano_len)), _ in (iterator):
            src, trg = intro.transpose(1,0), solo.transpose(1,0)
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg[:-1, :]) #turn off teacher forcing

            output = output.view(-1, output.shape[-1])
            trg = trg[1:].reshape(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.cpu().detach().item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



In [15]:
def translate_sentence(model, sentence, german, english, device, max_length=1200):

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    tokens = [token.lower() for token in sentence.split(' ')]
    # print(tokens)

    # sys.exit()
    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break
    # print(outputs)
    translated_sentence = [english.vocab.itos[idx] for idx in outputs]

    # remove start token
    return translated_sentence


In [16]:
df_intro = pd.read_csv(source_folder + '/val_torchtext.csv')
val_intro = df_intro['intro'].values
val_solo = df_intro['solo'].values
val_outro = df_intro['outro'].values
val_data=[]
for i in range(len(val_intro)):
    temp_dict = {}
    temp_dict['intro'] = val_intro[i]
    temp_dict['solo'] = val_solo[i]
    temp_dict['outro'] = val_outro[i]
    val_data.append(temp_dict)
print(len(val_intro))

112


In [17]:
def check_mode_collapse(model):
    count = 0
    translations = []
    for i in range(5):
        if len(val_intro) > 1200:
            continue
        intro = val_intro[i]
        solo = val_solo[i]
        outro = val_outro[i]
        #print(intro)
        list_intro = [int(x) for x in intro.split(' ')]
        list_solo = [int(x) for x in solo.split(' ')]
        list_outro = [int(x) for x in outro.split(' ')]
        translated_sentence = translate_sentence(model, intro, intro_field, solo_field, device, max_length=1200)
        
        translated_sentence = [int(x) for x in translated_sentence if x != '<pad>' and x != '<sos>' and x != '<eos>' and x != '<unk>']
        print(translated_sentence)
        translations.append(translated_sentence)
        if i > 0:
            if translations[i-1] == translations[i]:
                count += 1
    return count


In [18]:
N_EPOCHS = 500
S_EPOCH = 0
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(S_EPOCH, N_EPOCHS):
    
    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        checkpoint = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'valid_loss': valid_loss}
        save_best_checkpoint(checkpoint,N_EPOCHS)
    if (epoch+1) % 20 == 0 or (epoch) % 20 == 0:
        save_final_checkpoint(checkpoint,epoch)
    if (epoch+2) % 20 ==0:
        if check_mode_collapse(model) > 2:
            print("model is mode collapsing")
save_final_checkpoint(checkpoint,N_EPOCHS)
test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

KeyboardInterrupt: 

In [ ]:
# checkpoint = {'model_state_dict': model.state_dict(),
#                   'optimizer_state_dict': optimizer.state_dict(),
#                   'valid_loss': valid_loss}
# save_checkpoint(destination_folder + checkpoint,N_EPOCHS)

In [ ]:
best_model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
).to(device)
optimizer = optim.Adam(best_model.parameters(), lr=0.001)

In [19]:
state = torch.load(destination_folder + '/500_checkpoint.pt', map_location=device)
for key in state:
    print(key)
load_checkpoint(state, model, optimizer)

model_state_dict
optimizer_state_dict
valid_loss
=> Loading checkpoint


In [21]:
test_loss = evaluate(model, test_iter, criterion)
print(math.exp(test_loss))

256.0877787817241


In [25]:
df_intro = pd.read_csv(source_folder + '/test_torchtext.csv')
test_intro = df_intro['intro'].values
test_solo = df_intro['solo'].values
test_outro = df_intro['outro'].values
test_data=[]
for i in range(len(test_intro)):
    temp_dict = {}
    temp_dict['intro'] = test_intro[i]
    temp_dict['solo'] = test_solo[i]
    temp_dict['outro'] = test_outro[i]
    test_data.append(temp_dict)
print(len(test_intro))

112


In [26]:
for i in range(0,len(test_intro)):
    intro = test_intro[i]
    solo = test_solo[i]
    outro = test_outro[i]
    #print(intro)
    list_intro = [int(x) for x in intro.split(' ')]
    list_solo = [int(x) for x in solo.split(' ')]
    list_outro = [int(x) for x in outro.split(' ')]
    #print(list_sentence)
    translated_sentence = translate_sentence(model, intro, intro_field, solo_field, device, max_length=1200)
    #print(translated_sentence)
    translated_sentence = [int(x) for x in translated_sentence if x != '<pad>' and x != '<sos>' and x != '<eos>' and x != '<unk>']
    print(translated_sentence)
    utils.write_midi(list_intro, word2event, generated_outputs + "/intro/" + "/intro" + str(i)  + ".mid")
    utils.write_midi(list_solo, word2event, generated_outputs  + "/solo/" + "/solo" + str(i)  + ".mid")
    utils.write_midi(list_outro, word2event, generated_outputs + "/outro/" + "/outro" + str(i)  + ".mid")
    utils.write_midi(translated_sentence, word2event, generated_outputs + "/predict/" + "/predict" + str(i)  + ".mid")
    print(i)
    
        


[0, 1, 50, 127, 1, 57, 68, 6, 7, 4, 69, 6, 10, 17, 53, 12, 0, 1, 57, 53, 6, 37, 62, 69, 9, 7, 35, 68, 9, 42, 60, 69, 9, 10, 62, 68, 9, 43, 22, 32, 9, 49, 60, 52, 9, 28, 60, 32, 20, 13, 66, 68, 9, 31, 35, 52, 6, 15, 17, 30, 47, 36, 17, 30, 9, 0, 37, 17, 32, 6, 10, 17, 29, 44, 49, 17, 77, 9, 28, 57, 52, 9, 13, 60, 58, 6, 15, 57, 29, 6, 16, 35, 58, 20, 23, 60, 58, 9, 24, 60, 77, 44, 0, 1, 95, 29, 6, 7, 39, 58, 9, 42, 55, 29, 6, 10, 95, 29, 6, 49, 55, 5, 9, 28, 57, 5, 9, 13, 39, 5, 9, 31, 60, 105, 38, 16, 39, 52, 6, 36, 57, 105, 9, 0, 1, 57, 5, 9, 37, 95, 65, 9, 7, 57, 53, 20, 10, 39, 5, 9, 10, 55, 58, 6, 49, 55, 105, 9, 28, 95, 53, 6, 13, 95, 54, 20, 31, 40, 32, 44, 15, 57, 53, 20, 21, 95, 65, 9, 16, 55, 87, 9, 23, 55, 58, 20, 24, 17, 106, 9, 36, 62, 98, 20, 0, 1, 57, 65, 9, 37, 35, 58, 6, 42, 40, 29, 20, 10, 40, 58, 20, 10, 39, 29, 20, 49, 55, 58, 6, 49, 62, 98, 6, 13, 62, 58, 6, 31, 95, 11, 9, 15, 55, 105, 9, 21, 55, 5, 9, 16, 50, 171, 0, 1, 39, 11, 20, 37, 55, 105, 9, 7, 40, 5, 9, 42, 

[0, 1, 50, 172, 10, 66, 69, 9, 43, 94, 53, 20, 49, 94, 72, 48, 24, 35, 53, 103, 0, 13, 61, 106, 9, 31, 63, 72, 20, 15, 94, 53, 12, 23, 61, 69, 6, 24, 60, 53, 6, 0, 1, 94, 72, 44, 7, 63, 53, 103, 23, 60, 53, 9, 24, 94, 106, 120, 0, 13, 62, 53, 6, 15, 60, 69, 44, 23, 55, 53, 20, 24, 61, 69, 44, 0, 1, 60, 68, 20, 37, 61, 52, 25, 16, 61, 69, 9, 23, 60, 53, 20, 24, 60, 69, 120, 0, 13, 94, 68, 9, 31, 61, 52, 20, 15, 55, 77, 12, 23, 81, 52, 9, 24, 62, 68, 48, 0, 24, 55, 87, 6, 36, 114, 105, 9]
8
[0, 1, 2, 137, 10, 2, 137, 13, 2, 137, 21, 4, 11, 96, 16, 2, 137, 0, 1, 2, 137, 37, 17, 18, 25, 10, 2, 137, 28, 95, 78, 19, 13, 2, 137, 16, 2, 137, 36, 17, 29, 44, 0, 1, 2, 137, 37, 4, 29, 44, 42, 95, 30, 96, 10, 2, 137, 13, 2, 137, 21, 4, 33, 44, 23, 57, 34, 44, 36, 95, 53, 75, 0, 21, 39, 33, 38, 36, 95, 53, 38, 0, 42, 17, 64, 38, 28, 95, 54, 48, 0, 37, 62, 32, 9, 7, 55, 33, 9, 42, 95, 34, 27, 31, 40, 34, 44, 21, 95, 53, 44, 23, 95, 54, 44, 36, 95, 64, 74, 0, 21, 95, 34, 20, 36, 95, 34, 20, 0, 37, 57

[0, 1, 2, 137, 7, 40, 90, 6, 10, 2, 137, 10, 95, 30, 27, 13, 2, 137, 13, 35, 90, 44, 15, 60, 30, 38, 16, 2, 137, 24, 35, 90, 44, 0, 1, 2, 137, 1, 35, 30, 27, 10, 2, 137, 10, 55, 90, 44, 49, 62, 30, 47, 13, 2, 137, 15, 39, 90, 6, 16, 2, 137, 16, 17, 30, 56, 0, 1, 2, 137, 1, 40, 90, 74, 10, 2, 137, 13, 2, 137, 16, 2, 137]
20
[0, 1, 2, 137, 7, 40, 90, 6, 10, 2, 137, 10, 95, 30, 27, 13, 2, 137, 13, 35, 90, 44, 15, 60, 30, 38, 16, 2, 137, 24, 35, 90, 44, 0, 1, 2, 137, 1, 35, 30, 27, 10, 2, 137, 10, 55, 90, 44, 49, 62, 30, 47, 13, 2, 137, 15, 39, 90, 6, 16, 2, 137, 16, 17, 30, 56, 0, 1, 2, 137, 1, 60, 90, 74, 10, 2, 137, 13, 2, 137, 16, 2, 137]
21
[0, 1, 2, 137, 7, 40, 90, 6, 10, 2, 137, 10, 95, 30, 27, 13, 2, 137, 13, 35, 90, 44, 15, 60, 30, 38, 16, 2, 137, 24, 35, 90, 44, 0, 1, 2, 137, 1, 35, 30, 27, 10, 2, 137, 10, 55, 90, 44, 49, 62, 30, 47, 13, 2, 137, 15, 39, 90, 6, 16, 2, 137, 16, 17, 30, 56, 0, 1, 2, 137, 1, 55, 90, 74, 10, 2, 137, 13, 2, 137, 16, 2, 137]
22
[0, 1, 2, 137, 7, 40, 90,

[0, 1, 50, 150, 37, 61, 111, 44, 42, 61, 87, 46, 43, 61, 105, 46, 28, 94, 26, 38, 28, 61, 98, 46, 31, 66, 98, 44, 21, 66, 90, 44, 21, 61, 78, 38, 23, 61, 98, 44, 36, 94, 26, 44, 36, 61, 90, 38, 0, 37, 61, 98, 46, 42, 66, 90, 47, 42, 61, 98, 20, 28, 81, 26, 85, 28, 61, 78, 44, 31, 61, 98, 44, 21, 66, 90, 44, 23, 61, 98, 44, 36, 61, 26, 44, 0, 37, 61, 105, 44, 42, 94, 26, 44, 43, 61, 88, 46, 28, 94, 87, 25, 28, 61, 78, 44, 31, 61, 98, 44, 21, 66, 90, 44, 23, 94, 111, 44, 23, 60, 98, 44, 36, 63, 88, 38, 36, 61, 78, 85, 0, 42, 61, 177, 44, 43, 61, 133, 44, 28, 61, 169, 38, 21, 61, 164, 38, 36, 61, 98, 46, 36, 61, 69, 46, 0, 37, 66, 5, 44, 37, 66, 5, 44, 42, 66, 105, 44, 42, 66, 52, 44, 43, 61, 5, 44, 43, 61, 32, 44, 28, 94, 87, 44, 28, 94, 77, 44, 31, 61, 5, 46, 31, 61, 32, 46, 21, 66, 105, 47, 21, 66, 52, 47, 36, 61, 98, 44, 36, 61, 69, 44, 0, 37, 61, 5, 44, 37, 61, 32, 44, 42, 66, 105, 44, 42, 66, 52, 44, 43, 61, 5, 44, 28, 61, 32, 44, 31, 61, 87, 38, 21, 66, 98, 44, 36, 61, 98, 44, 36, 

[0, 1, 50, 171, 37, 95, 77, 75, 10, 50, 171, 49, 22, 69, 6, 28, 57, 68, 9, 13, 50, 171, 13, 57, 41, 20, 31, 40, 68, 9, 15, 40, 41, 9, 21, 4, 77, 47, 16, 50, 171, 23, 57, 69, 12, 0, 1, 50, 171, 37, 40, 68, 38, 10, 50, 171, 43, 40, 69, 9, 49, 35, 68, 9, 49, 62, 41, 20, 28, 62, 77, 9, 13, 50, 171, 13, 57, 72, 103, 16, 50, 171, 23, 4, 77, 47, 0, 1, 50, 171, 37, 57, 52, 48, 37, 40, 33, 46, 10, 50, 171, 10, 57, 69, 46, 28, 62, 53, 44, 13, 50, 171, 31, 62, 32, 38, 31, 62, 34, 12, 16, 50, 171, 16, 62, 72, 47, 36, 40, 54, 6, 0, 1, 50, 171, 37, 57, 72, 38, 10, 50, 171, 43, 4, 53, 9, 49, 57, 54, 20, 28, 57, 72, 20, 13, 50, 171, 13, 4, 106, 12, 21, 62, 69, 38, 16, 50, 171, 36, 62, 69, 6, 0, 1, 50, 171, 37, 4, 65, 38, 10, 50, 171, 43, 62, 64, 9, 49, 40, 106, 9, 28, 57, 65, 9, 13, 50, 171, 13, 57, 107, 96, 16, 50, 171, 24, 22, 106, 20, 36, 57, 65, 20, 0, 1, 50, 171, 1, 22, 107, 27, 10, 50, 171, 43, 40, 108, 6, 28, 40, 107, 20, 13, 50, 171, 13, 57, 107, 44, 15, 57, 65, 44, 16, 50, 171, 16, 57, 106, 2

...

[0, 1, 50, 121, 37, 60, 41, 6, 42, 60, 41, 9, 10, 50, 121, 10, 60, 41, 9, 43, 35, 41, 6, 28, 60, 68, 9, 13, 50, 121, 13, 60, 41, 9, 31, 55, 41, 47, 16, 50, 121, 23, 60, 78, 6, 36, 62, 77, 6, 0, 1, 50, 121, 37, 60, 41, 6, 42, 60, 41, 6, 10, 50, 121, 43, 35, 68, 9, 49, 60, 41, 9, 28, 62, 77, 9, 13, 50, 121, 13, 60, 78, 20, 31, 35, 78, 38, 16, 50, 121, 23, 66, 26, 44, 36, 66, 98, 44, 0, 1, 50, 121, 37, 66, 90, 6, 42, 60, 90, 6, 10, 50, 121, 43, 35, 90, 6, 28, 60, 26, 6, 13, 50, 121, 31, 60, 90, 47, 16, 50, 121, 23, 60, 26, 6, 36, 35, 87, 20, 0, 1, 50, 121, 1, 60, 88, 9, 37, 60, 78, 6, 42, 60, 78, 6, 10, 50, 121, 43, 55, 77, 20, 49, 60, 78, 20, 28, 60, 98, 20, 13, 50, 121, 13, 66, 90, 20, 31, 60, 90, 47, 16, 50, 121, 23, 55, 70, 44, 36, 35, 72, 44, 0, 1, 50, 121, 37, 60, 73, 38, 10, 50, 121, 13, 50, 121]
69
[0, 1, 50, 171, 37, 95, 77, 75, 28, 22, 78, 71, 36, 57, 69, 9, 0, 37, 40, 41, 38, 36, 57, 77, 6, 0, 37, 57, 68, 9, 7, 57, 41, 9, 42, 40, 77, 47, 43, 40, 77, 6, 28, 40, 41, 103, 0, 37, 4

[0, 1, 50, 150, 37, 61, 111, 44, 42, 61, 87, 46, 43, 61, 105, 46, 28, 94, 26, 38, 28, 61, 98, 46, 31, 66, 98, 44, 21, 66, 90, 44, 23, 61, 78, 38, 0, 7, 61, 26, 44, 42, 61, 98, 44, 43, 61, 98, 44, 28, 61, 98, 20, 13, 61, 90, 47, 0, 37, 61, 98, 44, 42, 66, 87, 19, 23, 40, 98, 44, 36, 61, 26, 85, 0, 42, 66, 98, 85, 21, 66, 87, 82, 0, 42, 61, 26, 44, 43, 61, 98, 44, 28, 61, 98, 44, 28, 61, 98, 44, 21, 66, 98, 44, 23, 94, 90, 44, 36, 60, 98, 44, 0, 42, 60, 98, 44, 43, 60, 87, 85, 0, 42, 94, 87, 44, 43, 61, 90, 85, 28, 81, 77, 38, 21, 66, 90, 44, 23, 61, 98, 44, 36, 60, 78, 27, 0, 42, 66, 77, 46, 43, 61, 98, 46, 28, 61, 69, 44, 28, 61, 69, 44, 31, 61, 98, 44, 21, 61, 90, 44, 23, 60, 77, 44, 36, 61, 98, 44, 0, 37, 61, 105, 44, 42, 66, 26, 6, 42, 61, 98, 44, 43, 61, 98, 44, 28, 61, 87, 25, 21, 57, 98, 44, 36, 61, 87, 85, 0, 43, 61, 98, 44, 28, 61, 105, 44, 21, 61, 90, 44, 23, 60, 87, 85, 0, 37, 61, 98, 44, 28, 61, 26, 44, 23, 61, 78, 44, 36, 61, 98, 44, 0, 42, 61, 69, 85, 28, 61, 26, 44, 43, 6

[0, 1, 50, 171, 37, 95, 77, 75, 10, 50, 171, 49, 22, 69, 6, 28, 57, 68, 9, 13, 50, 171, 13, 57, 41, 20, 31, 40, 68, 9, 15, 40, 41, 9, 21, 4, 77, 47, 16, 50, 171, 23, 57, 69, 12, 0, 1, 50, 171, 37, 40, 68, 38, 10, 50, 171, 43, 40, 69, 9, 49, 35, 68, 9, 49, 62, 41, 20, 28, 62, 41, 9, 13, 50, 171, 13, 57, 72, 103, 16, 50, 171, 23, 4, 77, 47, 0, 1, 50, 171, 37, 57, 52, 48, 37, 40, 33, 46, 10, 50, 171, 10, 57, 69, 46, 28, 62, 53, 44, 13, 50, 171, 31, 62, 32, 38, 31, 62, 34, 12, 16, 50, 171, 16, 62, 72, 47, 36, 40, 54, 6, 0, 1, 50, 171, 37, 57, 72, 38, 10, 50, 171, 43, 4, 53, 9, 49, 57, 54, 20, 28, 57, 72, 20, 13, 50, 171, 13, 4, 106, 12, 21, 62, 69, 38, 16, 50, 171, 36, 62, 69, 6, 0, 1, 50, 171, 37, 4, 65, 38, 10, 50, 171, 43, 62, 64, 9, 49, 40, 106, 9, 28, 57, 65, 9, 13, 50, 171, 13, 57, 107, 96, 16, 50, 171, 24, 57, 65, 20, 36, 57, 65, 20, 0, 1, 50, 171, 1, 22, 107, 27, 10, 50, 171, 43, 40, 108, 6, 28, 40, 107, 20, 13, 50, 171, 13, 57, 107, 44, 15, 57, 65, 44, 16, 50, 171, 16, 57, 106, 20

[0, 1, 50, 121, 37, 22, 70, 71, 23, 62, 70, 44, 36, 57, 68, 6, 0, 37, 62, 34, 71, 7, 60, 90, 9, 42, 62, 77, 9, 10, 35, 41, 9, 43, 22, 77, 44, 28, 55, 90, 44, 31, 57, 18, 96, 36, 62, 68, 44, 0, 37, 62, 64, 85, 21, 57, 72, 6, 23, 22, 64, 44, 36, 22, 72, 6, 0, 37, 62, 72, 74, 7, 55, 90, 9, 42, 57, 77, 9, 10, 35, 30, 6, 43, 57, 77, 9, 49, 63, 18, 9, 28, 35, 90, 48, 23, 40, 72, 46, 0, 1, 62, 106, 20, 1, 62, 72, 20, 37, 57, 70, 56, 49, 57, 72, 20, 28, 57, 106, 20, 13, 57, 107, 56, 24, 57, 68, 20, 24, 55, 69, 20, 36, 57, 53, 9, 0, 1, 35, 69, 20, 1, 60, 68, 20, 37, 22, 70, 6, 42, 35, 41, 6, 43, 57, 41, 20, 49, 62, 68, 20, 28, 62, 69, 9, 13, 55, 68, 20, 13, 35, 41, 20, 31, 57, 68, 12, 16, 57, 90, 9, 23, 22, 77, 6, 36, 62, 41, 9, 0, 1, 35, 68, 20, 37, 57, 69, 44, 42, 57, 69, 20, 10, 57, 70, 20, 43, 57, 72, 44, 28, 22, 70, 9, 13, 55, 72, 9, 31, 57, 64, 9, 15, 62, 72, 20, 15, 57, 70, 20, 21, 40, 72, 12, 36, 62, 34, 6, 0, 1, 57, 70, 76]
96
[0, 1, 50, 171, 37, 95, 77, 75, 10, 50, 171, 49, 22, 69, 6,

[0, 1, 50, 171, 37, 95, 77, 75, 10, 50, 171, 49, 22, 69, 6, 28, 57, 68, 9, 13, 50, 171, 13, 57, 41, 20, 31, 40, 68, 9, 15, 40, 41, 9, 21, 4, 41, 47, 16, 50, 171, 23, 57, 69, 12, 0, 1, 50, 171, 37, 40, 68, 38, 10, 50, 171, 43, 40, 69, 9, 49, 35, 68, 9, 49, 62, 41, 20, 28, 62, 41, 9, 13, 50, 171, 13, 57, 72, 103, 16, 50, 171, 23, 4, 77, 47, 0, 1, 50, 171, 37, 57, 52, 48, 37, 40, 33, 46, 10, 50, 171, 10, 57, 69, 46, 28, 62, 53, 44, 13, 50, 171, 31, 62, 32, 38, 31, 62, 34, 12, 16, 50, 171, 16, 62, 72, 47, 36, 40, 54, 6, 0, 1, 50, 171, 37, 57, 72, 38, 10, 50, 171, 43, 4, 53, 9, 49, 57, 54, 20, 28, 57, 72, 20, 13, 50, 171, 13, 4, 106, 12, 21, 62, 69, 38, 16, 50, 171, 36, 62, 69, 6, 0, 1, 50, 171, 37, 4, 65, 38, 10, 50, 171, 43, 62, 64, 9, 49, 40, 106, 9, 28, 57, 65, 9, 13, 50, 171, 13, 57, 107, 96, 16, 50, 171, 24, 22, 65, 20, 36, 57, 65, 20, 0, 1, 50, 171, 1, 22, 107, 27, 10, 50, 171, 43, 40, 108, 6, 28, 40, 107, 20, 13, 50, 171, 13, 57, 107, 44, 15, 57, 65, 44, 16, 50, 171, 16, 57, 106, 20

In [27]:
import mido
for i in range(11):
    intro = mido.MidiFile(generated_outputs + "/intro/" + '/intro' + str(i) + '.mid')
    solo = mido.MidiFile(generated_outputs + "/solo/" +'/solo' + str(i) + '.mid')
    outro = mido.MidiFile(generated_outputs + "/outro/" +'/outro' + str(i) + '.mid')
    predict = mido.MidiFile(generated_outputs + "/predict/" +'/predict' + str(i) + '.mid')
    total_intro_time = 0
    total_solo_time = 0
    total_predict_time = 0
    for msg in intro.tracks[1]:
        if msg.type == "note_on":
            total_intro_time += msg.time
    for msg in solo.tracks[1]:
        if msg.type == "note_on":
            total_solo_time += msg.time
    for msg in predict.tracks[1]:
        if msg.type == "note_on":
            total_predict_time += msg.time
            
    original_outro_time = 0 + outro.tracks[1][1].time
    
    print(original_outro_time + total_solo_time + total_intro_time)
    solo.tracks[1][1].time += total_intro_time
    outro.tracks[1][1].time = original_outro_time + total_solo_time + total_intro_time
    print(outro.tracks[1][1].time)
    intro.tracks[1].name = "intro"
    solo.tracks[1].name = "solo"
    outro.tracks[1].name = "outro"
    predict.tracks[1].name = "predict"
    merged_mid = mido.MidiFile()
    merged_mid.ticks_per_beat = intro.ticks_per_beat
    merged_mid.tracks = intro.tracks + solo.tracks + outro.tracks
    merged_mid.save(generated_outputs + '/merged' + str(i) + '.mid')
    
    
    outro = mido.MidiFile(generated_outputs + "/outro/" +'/outro' + str(i) + '.mid')
    
    print(original_outro_time + total_predict_time + total_intro_time)
    predict.tracks[1][1].time += total_intro_time
    outro.tracks[1][1].time = original_outro_time + total_predict_time + total_intro_time
    print(outro.tracks[1][1].time)
    merged_mid = mido.MidiFile()
    merged_mid.ticks_per_beat = intro.ticks_per_beat
    merged_mid.tracks = intro.tracks + predict.tracks + outro.tracks
    merged_mid.save(generated_outputs + '/merged_predict' + str(i) + '.mid')

25080
25080
27360
27360
29880
29880
24000
24000
23760
23760
18240
18240
35820
35820
24480
24480
22800
22800
23280
23280
20160
20160
23880
23880
25380
25380
25140
25140
25800
25800
36900
36900
25320
25320
23460
23460
40620
40620
24240
24240
25620
25620
18240
18240


In [ ]:
class BeamSearchNode(object):
    def __init__(self, prev_node, wid, logp, length):
        self.prev_node = prev_node
        self.wid = wid
        self.logp = logp
        self.length = length

    def eval(self):
        return self.logp / float(self.length - 1 + 1e-6)
# }}}
import copy
from heapq import heappush, heappop

In [ ]:
def translate_sentence_beam(model, sentence, german, english, device, max_length=1200,beam_width=2,max_dec_steps=25000):
    
    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    tokens = [token.lower() for token in sentence.split(' ')]
    # print(tokens)

    # sys.exit()
    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    eos_token = english.vocab.stoi["<eos>"]
    sos_token = english.vocab.stoi["<sos>"]
    
    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    
    n_best_list = []
    
     
    #trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

    #first token as input
    trg_tensor = torch.LongTensor(outputs).to(device)
    
    end_nodes = []

    #starting node
    node = BeamSearchNode(prev_node=None, wid=trg_tensor, logp=0, length=1)

    nodes = []

    heappush(nodes, (-node.eval(), id(node), node))
    n_dec_steps = 0

    while True:
        # Give up when decoding takes too long
        if n_dec_steps > max_dec_steps:
            break
        
        # Fetch the best node
        #print([n[2].wid for n in nodes])
        score, _, n = heappop(nodes)
        decoder_input = n.wid
        
        if n.wid.item() == eos_token and n.prev_node is not None:
            end_nodes.append((score, id(n), n))
            # If we reached maximum # of sentences required
            if len(end_nodes) >= beam_width:
                break
            else:
                continue
   
        sequence = [n.wid.item()]
        a = n
        while a.prev_node is not None:
            a = a.prev_node
            sequence.append(a.wid.item())
        sequence = sequence[::-1] # reverse
        
        #print(sequence)
        
        with torch.no_grad():
            output = model(sentence_tensor, torch.LongTensor(sequence).unsqueeze(1).to(device))
        
        # Get top-k from this decoded result
        topk_log_prob, topk_indexes = torch.topk(output, beam_width)
        #print(topk_indexes)
        #print(topk_log_prob)
        # Then, register new top-k nodes
        for new_k in range(beam_width):
            decoded_t = topk_indexes[0][0][new_k].view(1) # (1)
            logp = topk_log_prob[0][0][new_k].item() # float log probability val

            node = BeamSearchNode(prev_node=n,
                                  wid=decoded_t,
                                  logp=n.logp+logp,
                                  length=n.length+1)
            heappush(nodes, (-node.eval(), id(node), node))
        n_dec_steps += beam_width
        #print(n_dec_steps)
    # if there are no end_nodes, retrieve best nodes (they are probably truncated)
    if len(end_nodes) == 0:
        end_nodes = [heappop(nodes) for _ in range(beam_width)]

    # Construct sequences from end_nodes
    n_best_seq_list = []
    for score, _id, n in sorted(end_nodes, key=lambda x: x[0]):
        sequence = [n.wid.item()]
        # back trace from end node
        while n.prev_node is not None:
            n = n.prev_node
            sequence.append(n.wid.item())
        sequence = sequence[::-1] # reverse

        n_best_seq_list.append(sequence)


    # return n_best_seq_list

    translated_sentence = [english.vocab.itos[idx] for idx in n_best_seq_list[0]]

    # remove start token
    return translated_sentence


In [ ]:
def save_vocab(vocab, path):
    output = open(path, 'wb')
    pickle.dump(vocab, output)
    output.close()

In [ ]:
vocab_folder = "vocab/"
save_vocab(intro_field.vocab, vocab + '/intro_vocab.pkl')
save_vocab(solo_field.vocab, vocab  + '/solo_vocab.pkl')
save_vocab(outro_field.vocab, vocab + '/outro_vocab.pkl')

In [ ]:
def load_vocab(path):
    with open(path, 'rb') as f:
        x = pickle.load(f)
    return x

In [ ]:
vocab_folder = "vocab_intro/"
intro_field.vocab = load_vocab(vocab_folder + 'intro_vocab.pkl')
solo_field.vocab = load_vocab(vocab_folder + 'solo_vocab.pkl')
outro_field.vocab = load_vocab(vocab_folder + 'outro_vocab.pkl')

In [ ]:
vocab_folder = "vocab/"
with open()
intro_field.vocab = 

In [ ]:
beam = [2, 59, 119, 13, 212, 59, 212, 59, 59, 75, 59, 59, 13, 119, 59, 59, 59, 212, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 119, 59, 59, 59, 59, 166, 59, 59, 59, 13, 212, 59, 59, 59, 158, 59, 59, 59, 212, 59, 59, 59, 212, 212, 13, 59, 59, 59, 59, 212, 59, 212, 212, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 13, 59, 59, 59, 59, 59, 14, 59, 59, 212, 59, 212, 212, 59, 59, 59, 68, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 13, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 212, 212, 212, 59, 59, 59, 59, 59, 59, 68, 59, 59, 212, 59, 59, 13, 59, 59, 59, 59, 59, 59, 97, 59, 59, 59, 59, 212, 59, 59, 166, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 212, 212, 59, 59, 59, 59, 59, 59, 59, 158, 59, 59, 59, 59, 212, 59, 59, 59, 13, 59, 59, 59, 59, 158, 59, 59, 13, 59, 13, 59, 59, 59, 59, 212, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 13, 212, 13, 59, 59, 59, 59, 212, 59, 212, 59, 59, 59, 59, 59, 59, 212, 59, 59, 212, 59, 59, 59, 59, 13, 59, 59, 59, 158, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 212, 59, 59, 59, 59, 59, 59, 212, 158, 59, 59, 59, 212, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 13, 212, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 212, 59, 59, 158, 59, 212, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 212, 59, 59, 59, 212, 59, 13, 13, 59, 59, 13, 59, 212, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 13, 212, 59, 59, 59, 59, 59, 212, 59, 212, 59, 59, 212, 59, 59, 59, 212, 59, 212, 212, 59, 59, 59, 59, 59, 59, 13, 59, 166, 59, 212, 212, 59, 59, 59, 59, 59, 212, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 86, 59, 212, 212, 212, 59, 59, 59, 59, 212, 59, 86, 59, 59, 59, 212, 212, 212, 59, 59, 59, 59, 59, 59, 59, 13, 59, 59, 59, 13, 59, 59, 59, 59, 59, 59, 59, 59, 212, 212, 59, 59, 59, 59, 59, 212, 13, 59, 59, 59, 212, 59, 212, 59, 59, 166, 59, 86, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 166, 212, 59, 59, 59, 59, 59, 86, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 212, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 212, 13, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 13, 13, 59, 59, 212, 212, 158, 59, 59, 13, 212, 59, 59, 212, 59, 59, 59, 59, 212, 59, 59, 212, 59, 59, 212, 59, 158, 212, 59, 212, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 212, 59, 59, 212, 59, 212, 212, 59, 212, 59, 59, 59, 212, 59, 59, 212, 59, 59, 59, 13, 59, 212, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 158, 59, 59, 59, 212, 59, 212, 86, 59, 59, 59, 158, 212, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 212, 212, 59, 59, 59, 59, 212, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 212, 59, 212, 59, 13, 59, 59, 212, 59, 59, 59, 13, 59, 59, 59, 59, 59, 59, 13, 212, 59, 59, 59, 59, 59, 68, 59, 13, 59, 59, 13, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 14, 59, 59, 59, 59, 59, 59, 13, 86, 59, 59, 59, 212, 59, 86, 59, 59, 59, 59, 59, 59, 59, 212, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 212, 212, 59, 59, 59, 13, 59, 59, 59, 59, 68, 59, 59, 59, 212, 13, 59, 59, 59, 212, 59, 59, 212, 59, 59, 212, 59, 59, 212, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 13, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 212, 59, 59, 212, 59, 59, 59, 59, 59, 59, 13, 59, 59, 59, 212, 212, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 212, 212, 212, 13, 59, 166, 59, 212, 59, 59, 59, 13, 59, 59, 59, 59, 59, 59, 59, 59, 166, 212, 212, 59, 59, 212, 59, 212, 59, 59, 13, 59, 59, 59, 59, 13, 59, 59, 14, 13, 59, 59, 86, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 158, 59, 59, 59, 59, 212, 59, 59, 158, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 212, 59, 59, 59, 212, 59, 59, 212, 59, 59, 59, 212, 59, 59, 212, 59, 59, 59, 59, 212, 59, 59, 59, 59, 212, 59, 59, 59, 212, 59, 59, 59, 166, 59, 59, 59, 59, 59, 59, 59, 59, 13, 13, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 68, 59, 59, 59, 59, 59, 212, 212, 59, 59, 59, 59, 59, 59, 212, 59, 212, 59, 59, 59, 212, 59, 59, 13, 59, 59, 166, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 212, 59, 59, 59, 212, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 212, 158, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 158, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 212, 212, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 13, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 13, 59, 59, 59, 59, 59, 212, 59, 59, 212, 59, 59, 59, 59, 59, 212, 59, 59, 166, 59, 59, 59, 59, 13, 59, 59, 212, 212, 59, 59, 212, 59, 59, 59, 59, 59, 212, 59, 59, 59, 59, 212, 59, 59, 59, 59, 59, 59, 59, 212, 59, 212, 59, 59, 59, 59, 212, 59, 59, 59, 59, 212, 59, 59, 59, 59]

translated_sentence1 = [solo_field.vocab.itos[idx] for idx in beam]
translated_sentence = [int(x) for x in translated_sentence1 if x != '<pad>' and x != '<sos>' and x != '<eos>' and x != '<unk>']    
utils.write_midi(translated_sentence, word2event, generated_outputs + "/predict1.mid")

In [ ]:
translated_sentence

In [ ]:
for i in range(len(test_intro)):
    if len(test_intro) > 1200:
        continue
    list_sentence = [int(x) for x in sentence.split(' ')]
    remi = [word2event[x] for x in list_sentence]
    print(remi)

In [ ]:
def bleu_translate_sentence(model, sentence, german, english, device, max_length=1200):

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    #tokens = [token.lower() for token in sentence.split(' ')]
    # print(tokens)

    # sys.exit()
    # Add <SOS> and <EOS> in beginning and end respectively
    #tokens.insert(0, german.init_token)
    #tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    #text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(sentence).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]

    # remove start token
    return translated_sentence


In [ ]:
from torchtext.data.metrics import bleu_score

def bleu(data, model, german, english, device):
    targets = []
    outputs = []
    print(len(data))
    for example in data:
        #print( vars(example))
        src = vars(example)["intro"]
        trg = vars(example)["solo"]
        
        src = [int(x) for x in src]
        trg = [int(x) for x in trg]
        
        if len(trg) > 1200 or len(src) > 1200:
            continue
        
        prediction = bleu_translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append(trg)
        outputs.append(prediction)

    return bleu_score(outputs, targets)

In [ ]:
# running on entire test data takes a while
score = bleu(test[1:10], model, intro_field, solo_field, device)
print(f"Bleu score {score * 100:.2f}")

In [ ]:
# torch.backends.cudnn.enabled = False

In [ ]:
train_loss_list, valid_loss_list, global_steps_list = load_metrics(destination_folder + '/metrics.pt')
plt.plot(global_steps_list, train_loss_list, label='Train')
plt.plot(global_steps_list, valid_loss_list, label='Valid')
plt.xlabel('Global Steps')
plt.ylabel('Loss')
plt.legend()
plt.show() 

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns